# Pop2Piano - تجربة تدريب

شغّل الخلايا بالترتيب من 1 لآخر. آخر خلية تطلع ملف **training_report.txt** — حمّله وابعته لو حابب.

In [ ]:
# 1. تثبيت المكتبات
!pip install -q torch>=2.1.0 torchaudio>=2.1.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers>=4.40.0 lightning>=2.2.0 librosa soundfile pretty_midi omegaconf numba scipy pandas tqdm yt-dlp imageio-ffmpeg
!apt-get update -q && apt-get install -y -q ffmpeg
print("✅ تم التثبيت")

In [ ]:
# 2. استنساخ المشروع
import os
if os.path.exists("pop2piano"):
    %cd pop2piano
    !git pull origin main
else:
    !git clone -b main https://github.com/kareemkamal10/pop2piano.git
    %cd pop2piano
print("✅ المجلد:", os.getcwd())

In [ ]:
# 3. تحميل بيانات (من train_dataset.csv)
MAX_MB = 50
max_gb = MAX_MB / 1024
!python download/download.py train_dataset.csv output_dir/ --max_size_gb {max_gb} 2>&1 | tail -n 15
print("✅ انتهى التحميل")

In [ ]:
# 4. تشغيل التدريب
import os
DATA_DIR = "output_dir"
if not os.path.exists(DATA_DIR):
    print("❌ مفيش output_dir. شغّل خلية التحميل أولاً.")
else:
    !python train.py --data_dir {DATA_DIR} --precision 16-mixed --epochs 2 --batch_size 4 --lr 0.0001

In [ ]:
# 5. تقرير نهائي — حمّل الملف وابعته
import os
import sys
from datetime import datetime

report_path = "training_report.txt"
# لو شغّلت التقرير من /content، دور في pop2piano/output_dir
if os.path.exists("output_dir"):
    DATA_DIR = "output_dir"
elif os.path.exists("pop2piano/output_dir"):
    DATA_DIR = "pop2piano/output_dir"
else:
    DATA_DIR = None

lines = []
lines.append("=" * 50)
lines.append("Pop2Piano - تقرير تجربة التدريب")
lines.append("=" * 50)
lines.append(f"التاريخ: {datetime.now().isoformat()}")
lines.append(f"Python: {sys.version.split()[0]}")
lines.append(f"المجلد الحالي: {os.getcwd()}")
lines.append("")

if DATA_DIR and os.path.exists(DATA_DIR):
    all_items = os.listdir(DATA_DIR)
    subdirs = [x for x in all_items if os.path.isdir(os.path.join(DATA_DIR, x))]
    lines.append(f"عدد المجلدات (تراكات): {len(subdirs)}")
    valid = 0
    for name in subdirs[:20]:
        path = os.path.join(DATA_DIR, name)
        files = os.listdir(path)
        has_beat = any(f.endswith('.beatstep.npy') for f in files)
        has_notes = any(f.endswith('.notes.npy') for f in files)
        if has_beat and has_notes:
            valid += 1
        lines.append(f"  - {name}: جاهز للتدريب={has_beat and has_notes}, ملفات={len(files)}")
    if len(subdirs) > 20:
        lines.append(f"  ... و {len(subdirs)-20} مجلد آخر")
    lines.append(f"تراكات جاهزة للتدريب (من أول 20): {valid}")
else:
    lines.append("output_dir غير موجود (لو المجلد الحالي /content، شغّل كل الخلايا من الأول عشان تكون داخل pop2piano)")

lines.append("")
lines.append("انتهى التقرير — احفظ هذا الملف وابعته للمراجعة.")

report_text = "\n".join(lines)
with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_text)

print(report_text)
print(f"\n📄 الملف المحفوظ: {report_path}")